<a href="https://colab.research.google.com/github/mlindsey19/evol-comp-two/blob/master/EvolComp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import random
import numpy as np

N = 10 # must be even for crossover
RUNS = 30
LOCI = 3
LCBD = (-1.0, 5.0) #loci bounds
#print format -> #.###
float_formatter = lambda x: "%.8f" % x
np.set_printoptions( formatter = { 'float_kind' : float_formatter } )

gen = np.random.uniform( LCBD[ 0 ], LCBD[ 1 ], ( N, LOCI ) )


In [0]:
pen = 2
penFlag = 1

def fitness( individual ):
  fit = 0
  for x in individual:
    fit +=  pen * x**2 if ( penFlag and ( x > 5.0 or x < -1.0 ) ) else x**2
  return fit

def fitVector(g, n):
  fit = np.empty(n)
  for i in range(n):
    fit[ i ] = fitness( g[ i ] )
  return fit


def sortFit( g, n = N ):
  temp = np.c_[ g, fitVector( g, n ) ]
  return temp[ temp[ : , 3 ].argsort() ]


In [0]:
Pc = 0.8
alpha = 0.7
shuffFlag = 1

def crossover(  ):
  l = np.random.randint(0,3) # 0-2 (3 is exclusive)
  if ( shuffFlag ):
    np.random.shuffle( gen )
  for i in range( N // 2 ):
    if ( np.random.uniform( 0, 1 ) < Pc ):
      j = 2 * i
      gen[ j ][ l ], gen[ j + 1 ][ l ] = gen[ j + 1 ][ l ],  gen[ j ][ l ]
#  return gen

def arithmeticCrossover(  ):
  if ( shuffFlag ):
    np.random.shuffle( gen )
  for i in range( N // 2 ):
    l = np.random.randint(0,3) # 0-2 (3 is exclusive)
    if ( np.random.uniform( 0, 1 ) < Pc ):
      j = 2 * i
      x = gen[ j + 1 ][ l ] + gen[ j ][ l ]
      gen[ j ][ l ], gen[ j + 1 ][ l ] = ( 1 - alpha ) * x, alpha * x
  #    return gen

def arithmeticCrossPerLoci(  ):
  if ( shuffFlag ):
    np.random.shuffle( gen )
  for i in range( N // 2 ):
    for h in range( 3 ):        
      j = 2 * i
      if ( np.random.uniform( 0, 1 ) < Pc ):
        x = gen[ j + 1 ][ h ] + gen[ j ][ h ]
        gen[ j ][ h ], gen[ j + 1 ][ h ] = ( 1 - alpha ) * x, alpha * x
 # return gen
    
  
 

***mr*** mutation range
***ub*** upper bound

The upper bound is calculated so that numbers selected at random fall in an appropriate 
range of mutation ***P*** % of the time 

In [0]:
Pm = 0.1
mr = 0.75

def mut():
  mtmt = np.empty( ( N, LOCI) )
  ub = ( 2 * mr / Pm ) - mr
  for i in range ( N ):
    for j in range(LOCI):
      m = np.random.uniform( (0 - mr), ub )
      if ( m < mr ):
        mtmt[i][j] = m
      else:
        mtmt[i][j] = 0  
  return mtmt

def gausIndi():
  mtmt = np.empty( ( N, LOCI) )

  for i in range ( N ):
    sd = np.std(gen[i])
    for j in range(LOCI):
      m = np.random.uniform( 0 , 1 )
      if ( m < Pm ):
        mtmt[i][j] = np.random.normal(0, sd)
      else:
        mtmt[i][j] = 0   
  return mtmt

def gausPop():
  sd = np.std(gen)
  mtmt = np.random.normal( 0.0, sd, ( N, LOCI) )

  for i in range ( N ):
    for j in range(LOCI):
      m = np.random.uniform( 0 , 1 )
      if ( m > Pm ):
            mtmt[i][j] = 0  
  return mtmt

def mutation(  ):
  global gen
  gen = gen + mut()

def gpmutation(  ):
  global gen
  gen = gen + gausPop()
def gimutation(  ):
  global gen
  gen = gen + gausIndi()


def mutationWithClamp(  ):
  global gen
  ngen = gen + mut()
  for rw in range( N ):
    for cl in range( LOCI ):  
        ngen[rw][cl] = ngen[rw][cl] if ( ngen[rw][cl] > -1 ) else -1
        ngen[rw][cl] = ngen[rw][cl] if ( ngen[rw][cl] < 5 ) else 5
  gen = ngen

  



In [21]:
Psbt = 0.8

def selectionDetBT(  ):
  global gen
  ngen = np.empty( ( N, LOCI) )
  for i in range(N):
    x = gen[ random.randint( 0, N - 1 ) ]
    y = gen[ random.randint( 0, N - 1 ) ]
    if( fitness( x ) > fitness( y ) ):
      x = y
    ngen[ i ] = x
  gen = ngen
  
def selectionProbBT(  ):
  global gen
  ngen = np.empty( ( N, LOCI) )
  p = np.random.uniform( 0, 1 )
  for i in range(N):
    x = gen[ random.randint( 0, N - 1 ) ]
    y = gen[ random.randint( 0, N - 1 ) ]
    if( fitness( x ) > fitness( y ) and p < Psbt ):
      x = y
    ngen[ i ] = x
  gen = ngen
selectionProbBT()

print( gen )

[[-0.04938912 0.00086789 -0.00842052]
 [-0.04938912 0.04886104 -0.00842052]
 [-0.04938912 0.00086789 -0.00842052]
 [-0.04938912 0.00086789 -0.00842052]
 [-0.04938912 0.00086789 -0.00842052]
 [-0.04938912 0.00086789 -0.00842052]
 [-0.04938912 0.00086789 -0.00842052]
 [0.00463673 0.00086789 0.15376875]
 [-0.04938912 0.00086789 -0.00842052]
 [0.00463673 0.09204404 0.00717113]]


In [0]:
min = .5
max = 2 - min
Plr = lambda r:( min + ( r - 1 ) / ( N - 1 ) * ( max - min ) ) / N
PlrMap = np.array( [ Plr(x) for x in range( 1, 11 ) ] ) 
pmap =np.empty(N)
for i in range(N):
  if (i == 0):
    pmap[ i ] = PlrMap[ i ]
  else:
    pmap[ i ] = PlrMap[ i ] + pmap[ i - 1 ]


def selectionLR(  ):
  global gen
  ngen = np.empty( (N, LOCI) )
  temp = sortFit( gen )
  for i in range( N ):
    s = np.random.uniform( 0, 1 )
    for idx,c in enumerate(pmap):
      if( s <= c ):
        #print(s, c , 9 - idx)
        ngen[ i ] = temp[ 9-idx, : 3 ] 
        break
  gen = ngen

en = 2  # number of best fit guarantee propogate  

def selectionEliteLR(  ):
  global gen
  ngen = np.empty( (N, LOCI) )
  temp = sortFit( gen )
  for i in range( en ):
    ngen[ i ] = temp[ i, : 3]
  for i in range( en, N ):
    s = np.random.uniform( 0, 1 )
    for idx,c in enumerate(pmap):
      if( s <= c ):
        #print(s, c , 9 - idx)
        ngen[ i ] = temp[ 9-idx, : 3 ] 
        break
  gen = ngen

selectionLR()
print(gen)
selectionEliteLR()

print( gen )

In [15]:
tn = 8 # number to keep

firstRunFlag = 1
if ( firstRunFlag ):
  firstRunFlag = 0
  prevgen = np.random.uniform( LCBD[ 0 ], LCBD[ 1 ], ( N, LOCI ) )
  
def selectionTrunc(  ):
  global gen
  ngen = np.empty( ( N, LOCI ) )
  temp = sortFit( gen )
  for i in range( tn ):
    ngen[ i ] = temp[ i, : 3 ]
  for i in range ( tn, N ):
    ngen [ i ] = np.random.uniform( LCBD[ 0 ], LCBD[ 1 ], ( 1, LOCI ) )
  gen = ngen

def selectionFibTrunc(  ):
  global gen
  ngen = np.empty( ( N, LOCI ) )
  global prevgen
  temp = gen
  union = np.append(gen, prevgen, axis = 0)
  union = sortFit(union, n=(2 * N) )
  prevgen = temp
  for i in range( tn ):
    ngen[ i ] = union[ i, : 3 ]
  for i in range ( tn, N ):
    ngen [ i ] = np.random.uniform( LCBD[ 0 ], LCBD[ 1 ], ( 1, LOCI ) )
  gen = ngen

selectionFibTrunc()
print(gen)


[[0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.82278287 3.93605744 3.97061021]
 [2.34071517 1.07328978 3.25365569]]


In [0]:
for i in range(50):
  mutation()
  mutationWithClamp()
  arithmeticCrossPerLoci()
  arithmeticCrossover()
  arithmeticCrossPerLoci()
  crossover()
  selectionBT() 
  selectionTrunc() 
  selectionFibTrunc()
  selectionLR()
  selectionEliteLR()
print (gen)



In [30]:


mutList = [ gimutation, mutation, gpmutation]
crossList = [ arithmeticCrossover, arithmeticCrossPerLoci, crossover ]
selList = [ selectionDetBT, selectionProbBT, selectionTrunc, selectionFibTrunc, selectionLR, selectionEliteLR ]
avg = np.empty((3,3,6))
best = np.empty((3,3,6))
stdv = np.empty((3,3,6))
def evol():
  global gen
  global avg
  global best 
  global stdv
  bst = 0.0
  avgVector = np.empty(30)
  for c, cross in enumerate( crossList ):
    for m, mut in enumerate( mutList ):
      for s, sel in enumerate( selList ):
          for i in range( 30 ):
            gen = np.random.uniform( LCBD[ 0 ], LCBD[ 1 ], ( N, LOCI ) )
            for _ in range( 50 ):
              sel()
              cross()
              mut()
            bst = np.amin(fitVector(gen, N))
            avgVector[ i ] = np.mean(fitVector(gen, N))
          best[c][m][s] = bst if (bst < best[c][m][s] ) else best[c][m][s]
          avg[c][m][s] = np.mean(avgVector)
          stdv[c][m][s] = np.std(avgVector)



evol()

print( avg )
print( best )
print( stdv )

[[[0.00000904 0.00020289 6.11096334 4.26314729 0.00390654 0.00001230]
  [0.08148439 0.07271481 5.86197984 4.02505562 0.12677189 0.11301204]
  [0.00000575 0.00020280 7.80708128 4.27857430 0.00682567 0.00000366]]

 [[0.00000000 0.00000001 6.18558344 3.46544397 0.00000531 0.00000000]
  [0.07292094 0.07900347 6.14976642 3.44443550 0.12943559 0.08623640]
  [0.00000000 0.00000001 6.19621130 3.27887328 0.00002112 0.00000000]]

 [[0.00717961 0.09635131 8.76365938 5.29609951 0.12673100 0.00031124]
  [0.09059107 0.15779510 8.63289226 4.23969591 0.23459351 0.11580628]
  [0.00276313 0.00947979 11.79075297 4.24142601 0.14759724 0.00121422]]]
[[[0.00000000 0.00000422 0.49047032 0.00000000 0.00714571 0.00000004]
  [0.01100405 0.00237351 0.98893851 0.00001931 0.00403610 0.00008838]
  [0.00000015 0.00003953 0.17349556 0.00000143 0.00019026 0.00000065]]

 [[0.00000000 0.00000000 0.33920794 0.00000000 0.00000542 0.00000000]
  [0.00074883 0.00023603 0.35370734 0.00000227 0.00327701 0.00001984]
  [0.000000

**tn** -> number to keep during truncation selection

**min, max** -> (0,2) linear rank parameters

**en** -> number of top indivituals to keep for elite selection

**Psbt** -> prob that the best indiviual is kept in binary tournament 

**Pm** -> prob that loci is mutated

**mr** -> range (**-mr**,** mr**) value of mutation for uniform mutaion

**Pc**-> prob of crossover

**alpha** -> % distributed for arithmetic crossover

**shuffleflag** -> 1 = suffle before cross; 0 = no shuffle

In [0]:
tn = 8
min = .5
max = 2 - min
en = 2
Psbt = 0.8
Pm = 0.1
mr = 0.75
Pc = 0.8
alpha = 0.7
shuffFlag = 1